In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
import random
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
basepath = 'C:/Users/TRXMNT1337/Desktop/basepath/'
###train
train_img_directory = basepath + 'train/patches/img/'
train_msk_directory = basepath + 'train/patches_msk/img/' 

###test
test_img_directory = basepath + 'test/patches/img/'
test_msk_directory = basepath + 'test/patches_msk/img/'

valid_img_directory = basepath + 'test/patches/'
valid_msk_directory = basepath + 'test/patches_msk/'


###############################################################

train_img_list = os.listdir(train_img_directory)
print(len(train_img_list))
train_msk_list = os.listdir(train_msk_directory)
print(len(train_msk_list))

test_img_list = os.listdir(test_img_directory)
print(len(test_img_list))
test_msk_list = os.listdir(test_msk_directory)
print(len(test_msk_list))

In [ ]:
num_images = len(test_img_list)

img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(test_img_directory + test_img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot =cv2.imread(test_msk_directory + test_msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
%%time
train_images = []
train_masks = []
test_images = []
test_masks = []


#3 канальное потому, что segmentation models ожидают 3 канальные изображения
for img in os.listdir(train_img_directory):
    img = cv2.imread(train_img_directory + img, 1)# 1 значит читает изображение как RGB
    train_images.append(img)
            
train_images = np.array(train_images)

for img in os.listdir(train_msk_directory):
    img = cv2.imread(train_msk_directory + img, 0)# 0 начит читает изображение как grayscale
    train_masks.append(np.array(img))
            
train_masks = np.array(train_masks)

for img in os.listdir(test_img_directory):
    img = cv2.imread(test_img_directory + img, 1)
    test_images.append(np.array(img))
            
test_images = np.array(test_images)

for img in os.listdir(test_msk_directory):
    image = Image.open(test_msk_directory + img)
    test_masks.append(np.array(image))
            
test_masks = np.array(test_masks)

print(train_images.shape)
print(train_masks.shape)
print(test_images.shape)
print(test_masks.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder() #переводит значения 0 255 в 0 и 1

n, h, w = train_masks.shape

train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)

train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
y_train = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [ ]:
n, h, w = test_masks.shape

test_masks_reshaped = test_masks.reshape(-1,1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped)
test_masks_encoded_original_shape = test_masks_reshaped_encoded.reshape(n, h, w)
y_test = np.expand_dims(test_masks_encoded_original_shape, axis=3)

In [ ]:
#Reused parameters in all models

n_classes=1
activation='sigmoid'

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor

binary_focal_loss = sm.losses.BinaryFocalLoss(alpha=0.5, gamma=4.0)
binary_CE_loss = sm.losses.BinaryCELoss()
#dice_loss = sm.losses.DiceLoss(class_weights = np.array([0.05, 0.95]))

total_loss = binary_focal_loss # + dice_loss

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
y_train = np.float32(y_train)
y_test = np.float32(y_test)

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [ ]:
###Model 1
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(train_images)
X_test1 = preprocess_input1(test_images)

# define model
model1 = sm.Unet(BACKBONE1,
                 encoder_weights='imagenet',
                 encoder_freeze=False,
                 decoder_block_type = 'transpose',
                 classes=n_classes,
                 activation=activation)

# compile keras model with defined optimozer, loss and metrics
model1.compile(optimizer=optim,
               loss = total_loss,
               metrics=metrics)

#model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

print(model1.summary())

In [ ]:
filepath=basepath + 'new_models/' +  "saved_models/" + BACKBONE1 + "-{epoch:02d}-{val_iou_score:.2f}.hdf5" #File name includes epoch and validation accuracy.

early_stop = EarlyStopping(monitor='val_iou_score',
                           patience=7,
                           mode = 'max',
                           verbose=1)


#Use Mode = max for accuracy and min for loss. 
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_iou_score',
                             verbose=10,
                             save_best_only=True,
                             mode='max')

callbacks_list = [checkpoint]

In [ ]:
%%time

history1=model1.fit(X_train1, 
          y_train,
          batch_size=32, 
          epochs=100,
          verbose=1,
          validation_data=(X_test1, y_test),
        callbacks=callbacks_list)
##############################################

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training iou_score')
plt.plot(epochs, val_acc, 'r', label='Validation iou_score')
plt.title('Training and validation iou_score')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
valid_img_directory = basepath + 'test/patches/'
valid_msk_directory = basepath + 'test/patches_msk/'

valid_img_list = os.listdir(valid_img_directory)
print(len(valid_img_list))
valid_msk_list = os.listdir(valid_msk_directory)
print(len(valid_msk_list))

In [ ]:
num_images = len(valid_img_list)

img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(valid_img_directory + valid_img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot =cv2.imread(valid_msk_directory + valid_msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
%%time
valid_images = []

for img in os.listdir(valid_img_directory):
    if(os.path.isfile(valid_img_directory + img)):
        img = cv2.imread(valid_img_directory + img, 1)
        valid_images.append(np.array(img))

valid_images = np.array(valid_images)

from PIL import Image

valid_masks = []

for img in os.listdir(valid_msk_directory):
    if(os.path.isfile(valid_msk_directory + img)):
        image = Image.open(valid_msk_directory + img)
        valid_masks.append(np.array(image))

valid_masks = np.array(valid_masks)

In [ ]:
n, h, w = valid_masks.shape

valid_masks_reshaped = valid_masks.reshape(-1,1)
valid_masks_reshaped_encoded = labelencoder.fit_transform(valid_masks_reshaped)
valid_masks_encoded_original_shape = valid_masks_reshaped_encoded.reshape(n, h, w)
y_valid = np.expand_dims(valid_masks_encoded_original_shape, axis=3)

In [ ]:
BACKBONE = BACKBONE1
preprocess_input = sm.get_preprocessing(BACKBONE)

X_valid = preprocess_input(valid_images)
y_valid = np.float32(y_valid)

In [ ]:
model_name = 'resnet34-95-0.62.hdf5'
model = tf.keras.models.load_model(basepath + 'new_models/saved_models/' + model_name, compile = False)

In [ ]:
prediction = model.predict(X_valid)
y_pred_thresholded = prediction > 0.5

In [ ]:
intersection = np.logical_and(y_valid, y_pred_thresholded)
union = np.logical_or(y_valid, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print(" IoU socre is: ", iou_score)